# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902053


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:24,  2.93s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:09,  2.47s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:49,  1.82s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:35,  1.36s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:11<00:00,  4.59it/s]

Rendering models:  94%|█████████▎| 29/31 [00:11<00:00,  4.10it/s]

Rendering models:  97%|█████████▋| 30/31 [00:12<00:00,  3.97it/s]

Rendering models: 100%|██████████| 31/31 [00:12<00:00,  3.90it/s]

equidad1                              0.082396
not-logged-in-8c0c44faf754d344b18a    0.005243
ZigZagoon999                          0.078620
ElisabethB                            0.000296
Thedunkmasta                          0.001792
kebnekaise                            0.000222
FrancescoGullo                        0.000196
NikkiWalker                           0.000730
JEBala                                0.001353
Linda_J._Berkel                       0.000769
not-logged-in-fc6db6f8fca5399a40ba    0.044888
Paddy_van_CH                          0.000260
GabeGramy                             0.000565
lsautter                              0.000364
Jessica_Korkmaz                       0.002016
ericmalani                            0.000538
not-logged-in-468edef11712cfc1e24c    0.000195
Catdoesscience                        0.004595
acapirala                             0.000308
not-logged-in-fcfcbc88feb1aa0ed0a7    0.001270
Lavadude                              0.040967
clars915     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())